In [8]:
import csv

STOCKS_FILE_NAME = 'N5.csv'

def get_stocks():
    with open(STOCKS_FILE_NAME, 'r') as f:
        reader = csv.reader(f, delimiter=',', quotechar='"')
        stocks = [stock.strip() for row in reader for stock in row]
    return stocks

stocks = get_stocks()
stocks

['PIH',
 'FLWS',
 'FCCY',
 'SRCE',
 'VNET',
 'TWOU',
 'JOBS',
 'CAFD',
 'EGHT',
 'AVHI',
 'SHLM',
 'AAON',
 'ABAX',
 'ABEO',
 'ABEOW',
 'ABIL',
 'ABMD',
 'AXAS',
 'ACIU',
 'ACIA',
 'ACTG',
 'ACHC',
 'ACAD',
 'ACST',
 'AXDX',
 'XLRN',
 'ANCX',
 'ARAY',
 'ACRX',
 'ACET',
 'AKAO',
 'ACHN',
 'ACIW',
 'ACRS',
 'ACNB',
 'ACOR',
 'ATVI',
 'ACTA',
 'ACXM',
 'ADMS',
 'ADMP',
 'ADAP',
 'ADUS',
 'AEY',
 'IOTS',
 'ADMA',
 'ADBE',
 'ADTN',
 'ADRO',
 'AAAP',
 'ADES',
 'AEIS',
 'AMD',
 'ADXS',
 'ADXSW',
 'ADVM',
 'MAUI',
 'AEGN',
 'AGLE',
 'AEHR',
 'AMTX',
 'AERI',
 'AVAV',
 'AEZS',
 'AEMD',
 'GNMX',
 'AFMD',
 'AGEN',
 'AGRX',
 'AGYS',
 'AGIO',
 'AGNC',
 'AGNCB',
 'AGNCP',
 'AGFS',
 'AGFSW',
 'AIMT',
 'AIRM',
 'AIRT',
 'ATSG',
 'AIRG',
 'AMCN',
 'AKAM',
 'AKTX',
 'AKBA',
 'AKER',
 'AKRX',
 'AKTS',
 'ALRM',
 'ALSK',
 'AMRI',
 'ALBO',
 'ABDC',
 'ADHD',
 'ALDR',
 'ALDX',
 'ALXN',
 'ALCO',
 'ALGN',
 'ALIM',
 'ALJJ',
 'ALKS',
 'ABTX',
 'ALGT',
 'AIQ',
 'AHGP',
 'AMMA',
 'ARLP',
 'AHPI',
 'AMOT',
 'ALQA',


In [9]:
from datetime import datetime, timedelta
import pytz
import pandas as pd
import numpy as np
import requests

DATA_TIMEZONE = pytz.timezone("EST")

def get_trading_dates(today):
    """
    Returns the dates for the latest 10 trading days
    """
    url = "https://query1.finance.yahoo.com/v7/finance/chart/X?period1={:.0f}&period2={:.0f}"\
          "&interval=1d&includeTimestamps=true".format((today - timedelta(days=20)).timestamp(), today.timestamp())
    page = requests.get(url)
    chart = page.json()['chart']        

    result = chart['result'][0]
    start = datetime.fromtimestamp(result['timestamp'][-10], tz=DATA_TIMEZONE)
    end = datetime.fromtimestamp(result['timestamp'][-1], tz=DATA_TIMEZONE).replace(hour=16, minute=0)
    return start, end
            
get_trading_dates(datetime.now(tz=DATA_TIMEZONE).replace(hour=16, minute=0, second=0, microsecond=0))

(datetime.datetime(2017, 2, 28, 9, 30, tzinfo=<StaticTzInfo 'EST'>),
 datetime.datetime(2017, 3, 13, 16, 0, 6, tzinfo=<StaticTzInfo 'EST'>))

In [14]:
def get_data(symbol, start_date, end_date, raw_response=False):
        
    url = "https://query1.finance.yahoo.com/v7/finance/chart/{}?period1={:.0f}&period2={:.0f}"\
          "&interval=1m&indicators=quote&includeTimestamps=true&"\
          "events=div%7Csplit%7Cearn".format(symbol, start_date.timestamp(), end_date.timestamp())
    page = requests.get(url)
    chart = page.json()['chart']
        
    if chart['error']:
        print(chart['error'])
    
    if chart['result']:
        result = chart['result'][0]
        if raw_response:
            return result
        data = result['indicators']['quote'][0]
        data['time'] = [datetime.fromtimestamp(t, tz=DATA_TIMEZONE) for t in result['timestamp']]
        df = pd.DataFrame(data)
        return df[df.volume.notnull()]
   
today = datetime.now(tz=DATA_TIMEZONE).replace(hour=16, minute=0, second=0, microsecond=0)
start, end = get_trading_dates(today)
dd = get_data("GOOG", start, end)
dd

,close,high,low,open,time,volume
0,826.750000,826.750000,825.889954,825.889954,2017-02-28 09:30:00-05:00,0.0
1,827.454407,827.454407,827.440002,827.440002,2017-02-28 09:31:00-05:00,4741.0
2,828.309998,828.309998,828.309998,828.309998,2017-02-28 09:32:00-05:00,1880.0
3,827.529968,827.529968,827.529968,827.529968,2017-02-28 09:33:00-05:00,2263.0
4,827.390015,827.390015,826.620056,827.000000,2017-02-28 09:34:00-05:00,6427.0
5,826.986084,826.986084,826.986084,826.986084,2017-02-28 09:35:00-05:00,2376.0
6,827.385620,827.385620,827.385620,827.385620,2017-02-28 09:36:00-05:00,2316.0
7,826.270020,826.739990,826.270020,826.739990,2017-02-28 09:37:00-05:00,7823.0
8,826.019958,826.019958,826.010010,826.010010,2017-02-28 09:38:00-05:00,5079.0
9,826.250000,826.250000,826.000000,826.000000,2017-02-28 09:39:00-05:00,10569.0


In [11]:
avg9days = dd[dd.time.dt.date != end.date()].volume.mean()
print(avg9days)
dd = dd[dd.time.dt.date == end.date()].copy()
dd_len = len(dd)
dd['current_score'] = dd.volume / avg9days
dd['rolling_score'] = dd.current_score.rolling(window=dd_len, min_periods=1).mean()
dd

3647.5511734


,close,high,low,open,time,volume,current_score,rolling_score
3510,844.940002,845.200012,843.989990,843.989990,2017-03-13 08:30:00-05:00,36719.0,10.066754,10.066754
3511,844.110046,845.450012,844.110046,845.450012,2017-03-13 08:31:00-05:00,4614.0,1.264958,5.665856
3512,844.540039,844.540039,844.540039,844.540039,2017-03-13 08:32:00-05:00,1912.0,0.524187,3.951966
3513,845.404968,845.450012,844.815002,844.815002,2017-03-13 08:33:00-05:00,9628.0,2.639579,3.623870
3514,846.690002,846.690002,845.950012,845.950012,2017-03-13 08:34:00-05:00,8315.0,2.279612,3.355018
3515,847.719971,847.719971,846.979980,846.979980,2017-03-13 08:35:00-05:00,5886.0,1.613685,3.064796
3516,847.900024,847.900024,847.729980,847.820007,2017-03-13 08:36:00-05:00,7312.0,2.004633,2.913344
3517,847.139954,847.500061,846.690002,847.500061,2017-03-13 08:37:00-05:00,9254.0,2.537045,2.866307
3518,847.939941,847.939941,847.489990,847.489990,2017-03-13 08:38:00-05:00,6278.0,1.721155,2.739068
3519,846.620056,847.660034,846.620056,847.660034,2017-03-13 08:39:00-05:00,5381.0,1.475236,2.612684


In [ ]:
import threading
import queue


def get_data_parallel(symbols, start_date, end_date):    
    max_threads = 100
    
    def worker():
        while True:
            args = task_queue.get()
            if args is None:
                break
            response = get_data(*args, raw_response=True)
            if response:
                result_queue.put(response)
            task_queue.task_done()

    task_queue = queue.Queue()
    result_queue = queue.Queue()

    threads = []
    for i in range(min(max_threads, len(symbols))):
        t = threading.Thread(target=worker)
        t.start()
        threads.append(t)
    
    for stock in symbols:
        task_queue.put((stock, start_date, end_date))

    # block until all tasks are done
    task_queue.join()

    # stop workers
    for i in range(len(threads)):
        task_queue.put(None)

    for t in threads:
        t.join()
        
    # now process results
    df = None
    today = end_date.date()
    while True:
        try:
            result = result_queue.get(block=False)
        except queue.Empty:
            break
        else:
            if 'timestamp' in result:  # sometimes chart data is not available
                raw_data = result['indicators']['quote'][0]
                raw_data['time'] = [datetime.fromtimestamp(t, tz=DATA_TIMEZONE) for t in result['timestamp']]
                data = pd.DataFrame(raw_data)
                data['symbol'] = result['meta']['symbol']             

                avg9days = data[data.time.dt.date != today].volume.mean()
                data = data[data.time.dt.date == today].copy()
                dd_len = len(data)
                data['avg9days'] = avg9days
                data['current_score'] = data.volume / avg9days
                data['rolling_score'] = data.current_score.rolling(window=dd_len, min_periods=1).mean()

                data['time'] = data.time.dt.tz_convert(LOCAL_TZ)
                todays_data = data[data.time.dt.date == today]

                if df is None:
                    df = todays_data
                else:
                    df = df.append(todays_data, ignore_index=True)            
            
    return df[df.volume.notnull()]
   
today = datetime.now(tz=DATA_TIMEZONE).replace(hour=16, minute=0, second=0, microsecond=0)
LOCAL_TZ = 'America/Los_Angeles'
start, end = get_trading_dates(today)
all_dd = get_data_parallel(get_stocks(), start, end)
all_dd

In [ ]:
from matplotlib.finance import candlestick2_ochl
from matplotlib import pyplot as plt, ticker


def display_charts(df):
    df = df.reset_index()
    xticks = [n for n in range(0, len(df), 5)]
    
    # plot data
    fig = plt.figure(figsize=(18, 14)) 
    ax1 = plt.subplot(111)
    candlestick2_ochl(ax1, df.open, df.close, df.high, df.low, width=.9, colorup='g')
    ax1.set_xticks(xticks)
    ax1.set_xticklabels([df.loc[i].time.strftime('%H:%M') for i in ax1.get_xticks()], rotation=90)
    ax1.set_xlim(-1, len(df)+1)
    
    # Volume
    fig = plt.figure(figsize=(18, 5)) 
    ax2 = plt.subplot(111)
    positive = df[df.close >= df.open]
    negative = df[df.close < df.open]
    
    ax2.bar(positive.index, positive.volume, color='green', align='center', label="+Volume") 
    ax2.bar(negative.index, negative.volume, color='red', align='center', label="- Volume")
    ax2.set_xticks(xticks)
    ax2.set_xticklabels([df.time[i].strftime('%H:%M') for i in ax2.get_xticks()], rotation=90)
    ax2.set_xlim(-1, len(df)+1)
    # average volume
    ax2.plot(df.index, df.avg9days, label="9days AVG")
    ax2.legend(fancybox=True, loc='best')
    ax2.get_yaxis().set_major_formatter(ticker.FuncFormatter(lambda x,p: "%dK" % (x / 1000)))
    
    fig = plt.figure(figsize=(18, 5)) 
    ax3 = plt.subplot(111)
    ax3.bar(positive.index, positive.current_score, color='green', align='center', label="+Score")
    ax3.bar(negative.index, negative.current_score, color='red', align='center', label="- Score")    
    ax3.plot(df.index, df.rolling_score, label="Rolling Score")
    ax3.legend(fancybox=True, loc='best')
    ax3.set_xticks(xticks)
    ax3.set_xticklabels([df.time[i].strftime('%H:%M') for i in ax2.get_xticks()], rotation=90)
    ax3.set_xlim(-1, len(df)+1)
    
    plt.show()

display_charts(all_dd[all_dd.symbol == "GOOG"])

In [ ]:
from datetime import datetime, timedelta
from IPython.display import clear_output, display, HTML
import numpy as np
import pytz
from time import sleep

TOP_LIMIT = 10
LOCAL_TZ = 'America/Los_Angeles'
tz = pytz.timezone(LOCAL_TZ)
stocks = get_stocks()


def display_list_items(top_df, all_df):
    for n, item in enumerate(top_df.itertuples()):
        print(
            "{}. {} {}: Rolling Score {}, Current Score {}".format(
                n + 1, item.symbol, item.time.strftime("%I:%M%p"), 
                item.rolling_score, item.current_score
            )
        )
        display_charts(all_df[all_df.symbol == item.symbol])

last_time = None
start_date, end_date = None, None
while True:    
    now = datetime.now(tz=tz)
    start = now.replace(hour=6, minute=30, second=0, microsecond=0)
    stop = start.replace(hour=13, minute=0)
    
    if start <= now <= stop: # proper time for getting data
        
        if last_time: # no more often than once per minute
            diff = 60 - (now - last_time).seconds 
            if diff > 0:
                sleep(diff)
        
        if end_date is None or start_date is None:
            start_date, end_date = get_trading_dates(now)  
            
        df = get_data_parallel(stocks, start_date, end_date)
        
        if df is not None:
            clear_output()
            last_minute_data = df.groupby('symbol').last().reset_index()
            
            display(HTML("<h3>Rolling Score List</h3>"))           
            top_volimes = last_minute_data.sort_values(by='rolling_score', ascending=False)[:TOP_LIMIT]
            display_list_items(top_volimes, df)
            
            display(HTML("<h3>Current Score List</h3>"))
            top_volimes = last_minute_data.sort_values(by='current_score', ascending=False)[:TOP_LIMIT]
            display_list_items(top_volimes, df)                
         
        print("last update", now)
        print("prev update", last_time) 
        print("update time", datetime.now(tz=tz) - now)
        last_time = now        
    else:       
        start_date, end_date = None, None
        if now < start:
            diff = start - now
        else:
            start_tommorow = start + timedelta(days=1)
            diff = start_tommorow - now
            
        print("time to start:", diff)
        sleep(diff.seconds + diff.microseconds * 0.1 ** 6)